In [15]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime

In [42]:
loads_to_wh = pd.read_csv('data/roadnet/rel_to_wh/Loads created_concatenated.csv')

In [43]:
lines_per_file = int(len(loads_to_wh)/27)

In [44]:
filenum = 0
for i in range(len(loads_to_wh)):
    if i % lines_per_file == 0:
        filenum = filenum + 1
    #print(loads_to_wh.loc[i, "Load"])
    with open('data/roadnet/rel_to_wh/wh' + str(filenum) + '.csv', 'a') as fw:        
         fw.write(str(loads_to_wh.loc[i, "Load"])+'\n')